In [4]:
# !wget https://huggingface.co/datasets/ise-uiuc/Magicoder-OSS-Instruct-75K/resolve/main/data-oss_instruct-decontaminated.jsonl

In [5]:
import pandas as pd
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

In [6]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai7-aiservices-435901645.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-35-turbo'

In [25]:
instructions = []
with open('data-oss_instruct-decontaminated.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        instructions.append(l['problem'])

In [26]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [27]:
prompt = f'terjemah ke bahasa melayu: {instructions[0]}'

In [28]:
!mkdir data-oss_instruct-decontaminated

mkdir: cannot create directory ‘data-oss_instruct-decontaminated’: File exists


In [29]:
def generate(instruction, index):
    filename = f'data-oss_instruct-decontaminated/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        prompt = f'terjemah ke bahasa melayu: {instruction}'
        ins = predict(prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [30]:
generate(instructions[0], 0)

In [31]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range((len(instructions) // 6) * 0, (len(instructions) // 6) * 1, max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

100%|████████████████████████████████████| 6266/6266 [16:04:51<00:00,  9.24s/it]
